In [1]:
%run ../authentification.ipynb
import re
def xstr(s):
    if s is None:
        s="NoData"
        return s
    elif s=="":
        s="NoData"
        return s        
    else:
        s=str(s)
        s=re.sub('[^ a-zA-Z0-9]', '', s)
        #s=s.encode(encoding='UTF-8',errors='strict')
    return str(s).replace('"', '')

Authentification à l"api réussie
Connexion à neo4j réussie
Connexion à l"api plotly


In [ ]:
import tweepy
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        try:
            if status.lang =="en":

                if hasattr(status, 'extended_tweet'):
                    tweettext=status.extended_tweet['full_text']
                else:
                    tweettext=status.text

                if status.coordinates:
                    coordinatestweet=status.coordinates.coordinates
                else:
                    coordinatestweet="NoData"
                    
                if status.place:
                    placetweet=status.place.name
                else:
                    placetweet="NoData"

                if status.favorite_count:
                    favouritecounttweet=status.favorite_count
                else:
                    favouritecounttweet="NoData"
                    
                if status.user.created_at:
                    createdat=str(status.user.created_at)
                else:
                    createdat="NoData"

                hashtags=status.entities.get("hashtags")
                user_mentions=status.entities.get("user_mentions")
        
                sgraph.run('MERGE (tweet:Tweet {tid:"' + xstr(status.id) + '"'
                               ',source:"'+str(status.source)+'"'
                               ',date:"'+str(status.created_at)+'"'
                               ',coordinates:"'+coordinatestweet+'"'
                               ',place:"'+xstr(placetweet)+'"'
                               ',text: "'+xstr(status.text)+'"})'
                           'MERGE (user:User {id: "'+xstr(status.user.id)+'"'
                               ', screen_name: "'+xstr(status.user.screen_name)+'"'
                               ', url: "'+xstr(status.user.url)+'"'
                               ', description: "'+xstr(status.user.description)+'"'
                               ', name: "'+xstr(status.user.name) +'"'
                               ', created_at: "'+createdat+'"})'
                               ' CREATE unique (user)-[:Posted]->(tweet)')
                               
                sgraph.run('MATCH (t:Tweet) WHERE t.tid="'+xstr(status.id)+'" SET t.retweet_count = "'+str(status.retweet_count)+'", t.favorite_count="'+str(favouritecounttweet)+'"')
                sgraph.run('MATCH (u:User) WHERE u.id="'+xstr(status.user.id)+'" SET u.followers_count = "'+str(status.user.followers_count)+'", u.friends_count="'+str(status.user.friends_count)+'", u.statuses_count="'+str(status.user.statuses_count)+'"')
                
                for h in hashtags:
                        sgraph.run('MERGE (tweet:Tweet{tid: "'+xstr(status.id)+'"})'
                                       ' MERGE (h:Hashtag{tag: "'+xstr(h['text'])+'"})'
                                       'CREATE unique (tweet)-[:HAS]->(h)')

                for m in user_mentions:

                        sgraph.run(' MERGE (tweet:Tweet{tid: "'+xstr(status.id)+'"})'
                                    ' MERGE (m:User{MentionName: "'+xstr(m['name'])+'",mention_screen_name: "'+xstr(m['screen_name'])+'"})'
                                    'CREATE unique (tweet)-[:MENTIONS]->(m)')



        except tweepy.TweepError:
            print(tweepy.TweepError)
            sleep(60 * 15)
            print("on continue!")



     
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(track=['trumpKimSummit'],languages=['en'], stall_warnings=True)

    
class MyStreamListener(tweepy.StreamListener):

    def on_error(self, status_code):
        if status_code == 420:
            print("420")
            #returning False in on_error disconnects the stream
            return False

        # returning non-False reconnects the stream, with backoff.